## Traitement des données - Expérimentations (Modèle DIST_REL_C_RNN)

Après le constat que nous avons fait dans les notebooks 8.3, 8.3bis et 8.4 que les modèles DELTA_DIST+H, qui tentent de prédire les positions convergées des atomes produisent des résultats décevants à cause de la complexité de la tâche, nous allons construire un tout nouveau modèle (DIST_REL_C_RNN) qui devra prédire la distance entre deux atomes de carbones C1 et C2 partageant une liaison à partir des informations suivantes. Pour chaque atome en dehors des deux atomes de carbone :

* La distance avec les deux atomes de carbone
* La classe positionnelle de l'atome par rapport aux deux atomes de carbones. Pour la déterminer, on compare la  position de l'atome par rapport aux deux plans P1 (resp. P2) de vecteur normal C1C2 et passant par point C1 (resp. C2). Si l'atome est entre les deux plans il aura pour classe 'C', et s'il est derrière le plan P1 (resp. P2) il aura pour classe 'G' (resp 'D').
* Le numéro atomique de l'atome (encodé en one hot encoding)
* La masse atomique de l'atome.

Dans ce notebook, on va définir une méthode pour créer le jeu de données d'entrée du modèle à partir des données sur les coordonnées des atomes dans les molécules et leurs numéros et masses atomiques.



#### Chemins des fichiers

In [1]:
minimal_set_reduced = "../data/minimal_set_reduced.h5"

#### Calcul de la distance entre deux atomes

In [2]:
import numpy as np

def calcul_distance(pt1, pt2):
    """ Renvoie la distance entre deux points représentés par leurs coordonnées (x, y, z) dans deux tableaux
    de forme (1, 3)"""
    return np.sqrt(np.sum(np.square(np.diff(np.array([pt1, pt2]), axis=0))))

#### Fonction de test si les deux atomes de carbone partagent une liaison

Selon les mesures moyennes des distances carbone-carbone (http://hydra.vcp.monash.edu.au/modules/mod2/bondlen.html), on considère ici que deux atomes de carbone partagent une liaison si leur distance est comprise entre 1.15 Å et 1.60 Å.

In [3]:
def existe_liaison_C(coords_c1, coords_c2):
    dist = calcul_distance(coords_c1, coords_c2)
    return True
    return dist >= 1.15 and dist <= 1.54

#### Fonction renvoyant les distances d'un atome à deux autres atomes

In [4]:
def calcul_distances(pt1, pt_ref1, pt_ref2):
    return np.array([calcul_distance(pt1, pt_ref1), calcul_distance(pt1, pt_ref2)])

#### Fonction renvoyant la classe positionnelle d'un atome en fonction de sa position et des positions des deux atomes de référence. Renvoie le résultat sous forme de one-hot-encoding

In [30]:
def get_classe_pos(pt, c1, c2):
    
    # Initialisation du tableau de sortie
    classes_pos = np.zeros(shape=(3,))
    
    # Calcul du vecteur C1_C2
    vect = np.diff([c1, c2], axis=0)
    
    # Déclaration de la matrice contenant les coordonnées des trois points g, c et d
    gcd_pos = np.empty(shape=(3, 3))
    
    # Calcul du point c
    gcd_pos[1] = np.divide([np.sum([c1, c2], axis=0)], 2)

    # Calcul des points g et d
    gcd_pos[0] = gcd_pos[1] - vect
    gcd_pos[2] = gcd_pos[1] + vect
        
    # Calcul de la matrice des positions répétées du point
    pos = np.tile(pt, 3).reshape(3, 3)
    
    # Calcul des distances du point avec les points g, c et d
    dists = np.sqrt(np.sum(np.square(np.diff([gcd_pos, pos], axis=0)[0]), axis=1))
    
    # On renvoie le résultat encodé avec la méthode du one hot encoding
    classes_pos[np.argmin(dists)] = 1
    return classes_pos
    

#### Fonction renvoyant le numéro de l'atome encodé en one-hot encoding (pour les atomes des deux premières lignes du tableau périodique)

In [31]:
def get_anum_one_hot(z):
    
    if not 0 < z < 10:
        raise RuntimeError("Atomic number must be between 1 and 9")
    
    one_hot = np.zeros(shape=(9,))
    one_hot[int(z)-1] = 1
    return one_hot


#### Fonction renvoyant les données préparées pour le RN pour une molécule

In [61]:
def donnees_prep_RN_mol(coords_mol, anums_mol, amasses_mol):
    
    taille_mol = len(coords_mol)
    
    inputs_RN = []
    targets_RN = []
    
    # On itère sur tous les atomes de la molécule
    for i in range(taille_mol):
        
        # On ne s'intéresse qu'aux atomes de carbone
        if anums_mol[i] == 6.:
            
            # On parcourt tous les atomes suivants
            for j in range(i+1, taille_mol):
                
                # On ne s'intéresse qu'aux couples d'atomes de carbone partageant une liaison
                if anums_mol[j] == 6 and existe_liaison_C(coords_mol[i], coords_mol[j]):
                    
                    # On calcule la distance entre les deux atomes du couple
                    dist_c_c = calcul_distance(coords_mol[i], coords_mol[j])
                    
                    # On initialise l'entrée du RN pour le couple courant
                    input_rn = np.empty(shape=(taille_mol-2, 15))
                    
                    input_rn_idx = 0
                    
                    # On itère sur tous les atomes de la molécule (en dehors des deux atomes de carbone)
                    for k in range(taille_mol):
                        if k != i and k != j:
                            
                            # On enregistre le numéro et la masse atomique de la molécule
                            input_rn[input_rn_idx][:9] = get_anum_one_hot(anums_mol[k])
                            input_rn[input_rn_idx][9] = amasses_mol[k]
                            
                            # On enregistre les distances de l'atome aux deux atomes de carbone
                            input_rn[input_rn_idx][10:12] = calcul_distances(coords_mol[k], coords_mol[i],
                                                                         coords_mol[j])
                            
                            # On enregistre la classe positionnelle de l'atome par rapport aux deux atomes
                            # de carbone
                            input_rn[input_rn_idx][12:15] = get_classe_pos(coords_mol[k], coords_mol[i], 
                                                                        coords_mol[j])    

                            input_rn_idx += 1
                            
                    # On aplatit l'entrée du RN
                    input_rn = np.array(input_rn)
                    input_rn = input_rn.reshape(-1,)
                            
                    # On enregistre l'entrée et la cible du RN pour l'atome courant
                    inputs_RN.append(input_rn)
                    
                    # On enregistre la distance cible pour le couple d'atomes courant
                    targets_RN.append(dist_c_c)   
                    
        else:
            # Les carbones étant les premiers atomes de la molécule dans les données, on arrête la boucle
            # pour gagner du temps d'exécution
            break
            
    print(inputs_RN)
    
    return np.array(inputs_RN), np.array(targets_RN)
            
        


#### Test de la fonction de préparation des données sur une molécule fictive (on modifie temporairement la fonction qui détecte les liaisons par une fonction qui renvoie toujours True)

In [62]:
coords = np.array([[1., 1, 0],
                  [3., 1, 0],
                  [1., 2, 0],
                  [4., 2, 0],
                  [0., 4, 0],
                  [3., 3, 0],
                  [2., -1, 0]])

anums = np.array([6., 6, 6, 1, 1, 1, 1])
amasses = np.array([12., 12, 12, 1, 1, 1, 1])

inputs_rn, targets_rn = donnees_prep_RN_mol(coords, anums, amasses)
print(inputs_rn)
print(targets_rn)

[array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        , 12.        ,
        1.        ,  2.23606798,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        3.16227766,  1.41421356,  0.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        3.16227766,  4.24264069,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        2.82842712,  2.        ,  0.        ,  1.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1. 